In [2]:
import numpy as np
import nibabel as nib
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from scipy.spatial.distance import cdist

In [3]:
subject_id = "100206"
subject_path = f"diffusion_data/{subject_id}/T1w/Diffusion"
dwi_img = nib.load(f'{subject_path}/data.nii.gz')
mask_img = nib.load(f'{subject_path}/nodif_brain_mask.nii.gz')

In [4]:
# Convert to numpy arrays for processing
dwi_data = dwi_img.get_fdata()
mask = mask_img.get_fdata()

In [5]:
print(f"DWI data shape: {dwi_data.shape}")  # Should be (X, Y, Z, num_volumes)
print(f"Mask shape: {mask.shape}")          # Should be (X, Y, Z)

DWI data shape: (145, 174, 145, 288)
Mask shape: (145, 174, 145)


In [6]:
# Load gradient information (bvals and bvecs)
print("\nLoading gradient information...")
bvals, bvecs = read_bvals_bvecs(f'{subject_path}/bvals', 
                               f'{subject_path}/bvecs')
print(f"Number of gradient directions: {len(bvals)}")
print(f"bvals shape: {bvals.shape}")     # Should match number of volumes
print(f"bvecs shape: {bvecs.shape}")     # Should be (num_volumes, 3)


Loading gradient information...
Number of gradient directions: 288
bvals shape: (288,)
bvecs shape: (288, 3)


In [7]:
# Create gradient table for DIPY
gtab = gradient_table(bvals, bvecs)

In [8]:
# Identify and extract B0 (non-diffusion weighted) volumes
b0_mask = gtab.b0s_mask
b0_data = dwi_data[..., b0_mask]
print(f"\nNumber of B0 volumes: {np.sum(b0_mask)}")
print(f"B0 data shape: {b0_data.shape}")
# Average B0 volumes to get a single reference image
b0_avg = np.mean(b0_data, axis=-1)
print(f"Average B0 shape: {b0_avg.shape}")


Number of B0 volumes: 18
B0 data shape: (145, 174, 145, 18)
Average B0 shape: (145, 174, 145)


In [9]:
# Extract and normalize diffusion weighted volumes
dwi_mask = ~b0_mask  # Mask for diffusion weighted volumes
dwi_vols = dwi_data[..., dwi_mask]
print(f"\nNumber of DWI volumes: {np.sum(dwi_mask)}")
print(f"DWI volumes shape: {dwi_vols.shape}")


Number of DWI volumes: 270
DWI volumes shape: (145, 174, 145, 270)


In [10]:
# Normalize DWI volumes by B0 (avoid division by zero with small epsilon)
dwi_norm = dwi_vols / (b0_avg[..., None] + 1e-6)
print(f"Normalized DWI shape: {dwi_norm.shape}")

Normalized DWI shape: (145, 174, 145, 270)


In [11]:
# Find valid voxels using the brain mask
valid_idx = np.where(mask > 0)
print(f"\nNumber of valid voxels in mask: {len(valid_idx[0])}")


Number of valid voxels in mask: 936256


In [25]:
# Sample random voxels for training
n_samples = 50000  # Adjust this number based on your needs
sample_idx = np.random.choice(len(valid_idx[0]), 
                            min(n_samples, len(valid_idx[0])), 
                            replace=False)

In [26]:
# Extract features (signal intensities) from sampled voxels
features = []
for idx in sample_idx:
    x, y, z = valid_idx[0][idx], valid_idx[1][idx], valid_idx[2][idx]
    features.append(dwi_norm[x, y, z, :])

In [ ]:
features = np.array(features)
gradient_directions = bvecs[dwi_mask]  # Only keep directions for DWI volumes

In [ ]:
print("\nFinal data shapes:")
print(f"Features shape: {features.shape}")           # Should be (n_samples, n_directions)
print(f"Gradient directions shape: {gradient_directions.shape}")  # Should be (n_directions, 3)

# Basic sanity checks
print("\nSanity checks:")
print(f"Max normalized value: {np.max(features)}")
print(f"Min normalized value: {np.min(features)}")
print(f"Gradient directions magnitude close to 1: {np.allclose(np.linalg.norm(gradient_directions, axis=1), 1, atol=1e-3)}")


Final data shapes:
Features shape: (1000, 270)
Gradient directions shape: (270, 3)

Sanity checks:
Max normalized value: 425303588.8671875
Min normalized value: 0.0
Gradient directions magnitude close to 1: True


In [ ]:
print("Loading ground truth data...")
gt_data = np.load('ground_truth.npz')
ground_truth_tensors = gt_data['tensors']
valid_coordinates = gt_data['coordinates']
print(f"Ground truth tensors shape: {ground_truth_tensors.shape}")
print(f"Valid coordinates shape: {valid_coordinates.shape}")

Loading ground truth data...
Ground truth tensors shape: (936256, 6)
Valid coordinates shape: (936256, 3)


In [ ]:
# Number of directions to use for sparse estimation
n_directions = 41  # We can adjust this number

def create_graph_data(features, gradient_directions, n_sparse_directions, batch_size = 32, threshold_angle = 45):
    """
    Create graph data for a batch of samples, including ground tensor
    """
    voxel_indices = np.random.choice(len(features), size = batch_size, replace = False)
    print(f"Creating graphs for batch of {batch_size} voxels using {n_sparse_directions} directions each")
    
    batch_nodes = []
    batch_edges = []
    batch_tensors = []

    for idx in voxel_indices:
        # Randomly selected directions
        selected_dir_idx = np.random.choice(len(gradient_directions), size=n_sparse_directions, replace=False)

        directions = gradient_directions[selected_dir_idx]
        signals = features[idx, selected_dir_idx]
        
        # Create node features: x, y, z, signal
        nodes = np.column_stack([directions, signals])

        # Create edges acc. to angle
        directions_norm = directions / np.linalg.norm(directions, axis=1, keepdims=True)
        cos_sim = np.dot(directions_norm, directions_norm.T)
        angles = np.arccos(np.clip(cos_sim, -1.0, 1.0)) * 180/np.pi
        
        src, dst = np.where(angles < threshold_angle)
        mask = src != dst
        edges = np.column_stack([src[mask], dst[mask]])

        batch_nodes.append(nodes)
        batch_edges.append(edges)
        batch_tensors.append(ground_truth_tensors[idx])

        if idx == voxel_indices[0]:
            print("\nFirst sample edge check:")
            print(f"Total edges created: {edges.shape[0]}")
            print("Example edges and their angles:")
            for i in range(min(5, len(edges))):
                e1, e2 = edges[i]
                print(f"Edge {i}: {e1}->{e2}, Angle: {angles[e1,e2]:.2f} degrees")
        
    return batch_nodes, batch_edges, batch_tensors

nodes, edges, gt_tensors = create_graph_data(features, gradient_directions, n_directions)
# Print info for verification
print("\nBatch statistics:")
print(f"Number of samples in batch: {len(nodes)}")
print(f"Number of nodes per graph: {nodes[0].shape[0]}")
print(f"Node feature dimensionality: {nodes[0].shape[1]}")
print(f"Ground truth tensors: {len(gt_tensors)}")

# Print example for first sample
print("\nFirst sample in batch:")
print(f"Number of nodes: {nodes[0].shape[0]}")
print(f"Number of edges: {edges[0].shape[0]}")
print(f"Ground truth tensor components: {gt_tensors[0]}")

Creating graphs for batch of 32 voxels using 41 directions each

First sample edge check:
Total edges created: 348
Example edges and their angles:
Edge 0: 0->3, Angle: 28.07 degrees
Edge 1: 0->5, Angle: 12.20 degrees
Edge 2: 0->6, Angle: 33.02 degrees
Edge 3: 0->17, Angle: 23.30 degrees
Edge 4: 0->19, Angle: 37.34 degrees

Batch statistics:
Number of samples in batch: 32
Number of nodes per graph: 41
Node feature dimensionality: 4
Ground truth tensors: 32

First sample in batch:
Number of nodes: 41
Number of edges: 348
Ground truth tensor components: [ 4.26222250e-04 -2.06334230e-05  3.99538937e-04 -4.55641156e-06
 -5.93979584e-05  4.22905867e-04]
